In [20]:
#Importamos pandas y leemos la hoja puerto de nuestro archivo excel
import pandas as pd
df_puerto = pd.read_excel("Bd_Itinerarios.xlsx",sheet_name="puerto")

In [21]:
df_puerto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061 entries, 0 to 1060
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CodigoPuerto    1061 non-null   object
 1   CodigoPais      1060 non-null   object
 2   NombrePuerto    1061 non-null   object
 3   TipoTransporte  1061 non-null   object
 4   EsPuertoLocal   1061 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 41.6+ KB


In [22]:
df_puerto.describe()

,EsPuertoLocal
count,1061.000000
mean,0.004713
std,0.068518
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [23]:
df_puerto.isnull().sum()

CodigoPuerto      0
CodigoPais        1
NombrePuerto      0
TipoTransporte    0
EsPuertoLocal     0
dtype: int64

In [24]:
df_puerto[df_puerto.isnull().any(axis=1)].index

Index([1013], dtype='int64')

In [25]:
df_puerto.loc[1013]

CodigoPuerto             WVB
CodigoPais               NaN
NombrePuerto      WALVIS BAY
TipoTransporte             M
EsPuertoLocal              0
Name: 1013, dtype: object

In [26]:
#Eliminamos ya que el puerto no cuenta con un codigo de país por ende no se puede identificar al pais que pertence
df_puerto.drop(1013,inplace=True)

In [27]:
df_puerto.duplicated().sum()

np.int64(0)

In [28]:
df_puerto.duplicated(subset=["CodigoPuerto"]).sum()

np.int64(114)

In [29]:
df_puerto.duplicated(subset=["CodigoPuerto","CodigoPais","NombrePuerto"]).sum()

np.int64(50)

In [30]:
tuplas = list(df_puerto[["CodigoPuerto", "NombrePuerto"]].itertuples(index=False, name=None))
print(tuplas)

[(123, 'KOPER'), (827, 'HAYDARPASA'), ('AAE', 'ANNABA'), ('AAH', 'AACHEN'), ('AAL', 'AALBORG'), ('AAN', 'AL AIN'), ('AAR', 'AARHUS'), ('ABD', 'ABERDEEN'), ('ACB', 'ALBOLOTE'), ('ACC', 'ACCRA'), ('ACE', 'LANZAROTE'), ('ACI', 'TACOMA'), ('ACJ', 'ACAJUTLA'), ('ADA', 'ADANA'), ('ADD', 'ADDIS ABABA'), ('ADE', 'ADEN'), ('ADH', 'ANGRA DO HEROISMO'), ('ADL', 'ADELAIDE'), ('AES', 'ALESUND'), ('AGB', 'AUGSBURG'), ('AGP', 'MALAGA'), ('AGP', 'MALAGA'), ('AIN', 'HUAIYIN'), ('AJM', 'AJMAN'), ('AKL', 'ASHKELON'), ('AKL', 'AUCKLAND'), ('AKL', 'AUCKLAND'), ('ALB', 'ALBACETE'), ('ALC', 'ALICANTE'), ('ALC', 'ALICANTE'), ('ALE', 'ALESSANDRIA'), ('ALG', 'ALGER'), ('ALG', 'ALGECIRAS'), ('ALY', 'ALEXANDRIA'), ('AMB', 'AMBARLI'), ('AMD', 'AHMEDABAD'), ('AMM', 'AMMAN'), ('AMS', 'AMSTERDAM'), ('AMS', 'AMSTERDAM'), ('ANF', 'ANTOFAGASTA'), ('ANK', 'ANKARA'), ('ANR', 'ANTWERP'), ('ANS', 'ANSBACH'), ('AOI', 'ANCONA'), ('AOT', 'AOSTA'), ('APW', 'APIA'), ('AQG', 'ANQING'), ('ARE', 'AREZZO'), ('ARI', 'ARICA'), ('ASH',

In [31]:
df_puerto[df_puerto.duplicated(subset=["CodigoPuerto","CodigoPais","NombrePuerto"],keep=False)]

,CodigoPuerto,CodigoPais,NombrePuerto,TipoTransporte,EsPuertoLocal
20,AGP,ES,MALAGA,A,0
21,AGP,ES,MALAGA,M,0
25,AKL,NZ,AUCKLAND,A,0
26,AKL,NZ,AUCKLAND,M,0
28,ALC,ES,ALICANTE,A,0
...,...,...,...,...,...
972,UIO,EC,QUITO,M,0
988,VLC,ES,VALENCIA,A,0
989,VLC,ES,VALENCIA,M,0
1054,ZRH,CH,ZURICH,A,0


In [32]:
# Eliminar todos los espacios en blanco de los nombres de puerto para la generacion de nuevos codigos de puerto
df_puerto["NombreAntiguo"] = df_puerto['NombrePuerto']
df_puerto['NombrePuerto'] = df_puerto['NombrePuerto'].astype(str).str.replace(' ', '').str.upper()

In [33]:
from collections import defaultdict
# Guardar código antiguo 
df_puerto['CodAntiguo'] = df_puerto.get('CodigoPuerto', None)

# Funcion para generar codigo tomando en cuenta la primera letra , 2 letras del medio y la ultima letra
def generar_codigo_base(nombre):
    nombre = nombre.strip().upper()
    longitud = len(nombre)

    if longitud >= 4:
        medio1 = (longitud - 1) // 2
        medio2 = medio1 + 1 if medio1 + 1 < longitud else medio1
        return nombre[0] + nombre[medio1] + nombre[medio2] + nombre[-1]
    else:
        # Si el nombre es corto, rellenar con 'X'
        return (nombre + 'XXXX')[:4]

# Generar columna auxiliar
df_puerto['CodigoBase'] = df_puerto['NombrePuerto'].apply(generar_codigo_base)
nombre_a_codigo = {}  # (nombre, pais) → código asignado
codigos_usados = defaultdict(set)  # base → set de claves (nombre, pais)
contador = defaultdict(int)
codigos_finales = []

for _, row in df_puerto.iterrows():
    base = row['CodigoBase']
    nombre = row['NombrePuerto'].strip().upper()
    pais = row['CodigoPais'].strip().upper()
    clave = (nombre, pais)

    if clave in nombre_a_codigo:
        # Ya se asignó este nombre+país antes → reutiliza código
        codigos_finales.append(nombre_a_codigo[clave])
    else:
        if codigos_usados[base]:
            # Ese base ya fue usado para otro nombre o país → colisión
            contador[base] += 1
            nuevo_codigo = f"{base}{contador[base]}"
            codigos_finales.append(nuevo_codigo)
            nombre_a_codigo[clave] = nuevo_codigo
        else:
            # Primer uso de ese base
            codigos_finales.append(base)
            nombre_a_codigo[clave] = base

        codigos_usados[base].add(clave)

In [34]:
# Asignar código final
df_puerto['CodigoPuerto'] = codigos_finales
df_puerto.drop(columns=['CodigoBase'], inplace=True)

In [35]:
print(df_puerto)

     CodigoPuerto CodigoPais NombrePuerto TipoTransporte  EsPuertoLocal  \
0            KPER         SI        KOPER              M              0   
1            HARA         TR   HAYDARPASA              M              0   
2            ANAA         DZ       ANNABA              M              0   
3            ACHN         DE       AACHEN              M              0   
4            ABOG         DK      AALBORG              M              0   
...           ...        ...          ...            ...            ...   
1056         ZGSN         CN    ZHONGSHAN              M              0   
1057         ZUHI         CN       ZHUHAI              M              0   
1058         ZCKU         DE      ZWICKAU              M              0   
1059         XGAG         CN      XINGANG              M              0   
1060         ZGZU         CN    ZHANGZHOU              M              0   

     NombreAntiguo CodAntiguo  
0            KOPER        123  
1       HAYDARPASA        827  
2  

In [36]:
df_puerto[df_puerto.duplicated(subset=["CodigoPuerto","CodigoPais"])].index

Index([  21,   26,   29,   38,   56,   68,   71,   75,   82,   96,  105,  122,
        127,  134,  143,  149,  207,  260,  275,  308,  335,  365,  397,  409,
        435,  456,  541,  552,  565,  582,  585,  591,  596,  613,  625,  642,
        672,  720,  767,  794,  816,  830,  836,  841,  851,  855,  903,  922,
        938,  950,  963,  972,  989, 1036, 1038, 1055],
      dtype='int64')

In [37]:
columnas = ["CodigoPuerto","CodigoPais"]
fila_objetivo = df_puerto.loc[21,columnas]
coincidencias = df_puerto[columnas].eq(fila_objetivo).all(axis=1)
indices_coincidentes = df_puerto[coincidencias].index.tolist()
print(indices_coincidentes)

[20, 21]


In [38]:
df_puerto.loc[[20,21]]

,CodigoPuerto,CodigoPais,NombrePuerto,TipoTransporte,EsPuertoLocal,NombreAntiguo,CodAntiguo
20,MLAA,ES,MALAGA,A,0,MALAGA,AGP
21,MLAA,ES,MALAGA,M,0,MALAGA,AGP


In [39]:
df_puerto.drop_duplicates(subset=["CodigoPuerto", "CodigoPais"], keep="first", inplace=True)

In [40]:
df_puerto.to_csv("Puerto_silver.csv",index=False)


In [41]:
df_puerto = df_puerto.drop(["NombrePuerto",],axis=1)

In [42]:
df_puerto[df_puerto.duplicated(subset=["CodigoPuerto","CodigoPais"],keep=False)]

,CodigoPuerto,CodigoPais,TipoTransporte,EsPuertoLocal,NombreAntiguo,CodAntiguo


In [43]:
df_puerto = df_puerto.drop(["CodAntiguo",],axis=1)

In [44]:
df_puerto=df_puerto.rename(columns={"NombreAntiguo": "NombrePuerto"})

In [45]:
df_puerto

,CodigoPuerto,CodigoPais,TipoTransporte,EsPuertoLocal,NombrePuerto
0,KPER,SI,M,0,KOPER
1,HARA,TR,M,0,HAYDARPASA
2,ANAA,DZ,M,0,ANNABA
3,ACHN,DE,M,0,AACHEN
4,ABOG,DK,M,0,AALBORG
...,...,...,...,...,...
1056,ZGSN,CN,M,0,ZHONGSHAN
1057,ZUHI,CN,M,0,ZHUHAI
1058,ZCKU,DE,M,0,ZWICKAU
1059,XGAG,CN,M,0,XINGANG


In [46]:
df_puerto.to_csv("Puerto_Gold.csv",index=False)